# **Chapter 16: Selenium WebDriver**

---

## **16.1 Introduction to Selenium WebDriver**

### **What is Selenium WebDriver?**

**Selenium WebDriver** is an open-source tool that automates web browsers for testing purposes. It provides a programming interface to create and execute test cases that interact with web elements, simulate user actions, and verify application behavior across different browsers and platforms.

Unlike its predecessor Selenium RC (Remote Control), WebDriver communicates directly with the browser using native support—making it faster, more stable, and closer to how real users interact with applications.

**Formal Definition:**
> "Selenium WebDriver is a collection of open-source APIs used to automate the testing of web applications. It supports multiple programming languages and browsers, providing a unified interface for web automation." — Selenium Documentation

### **Why Selenium is the Industry Standard**

1. **Cross-Browser Support:** Chrome, Firefox, Edge, Safari, Internet Explorer
2. **Multi-Language Support:** Java, Python, C#, Ruby, JavaScript
3. **Platform Independence:** Windows, macOS, Linux
4. **Community & Ecosystem:** Largest automation community, extensive documentation
5. **W3C Standard:** Official web standard for browser automation
6. **Integration:** Works with CI/CD tools, TestNG/JUnit, Cucumber, etc.

### **Selenium Evolution**

```
Selenium 1.0 (2004) - Selenium RC
    └── JavaScript-based, required server, slow
        ↓
Selenium 2.0 (2011) - WebDriver Merge
    └── Native browser communication, faster
        ↓
Selenium 3.0 (2016) - Standalone Server
    └── GeckoDriver for Firefox, improved stability
        ↓
Selenium 4.0 (2021) - W3C Standard, Chrome DevTools
    └── Native CDP support, relative locators, new window/tab management
        ↓
Selenium 4.x (Current) - BiDi, CDP improvements
    └── Bidirectional communication, better DevTools integration
```

---

## **16.2 Selenium WebDriver Architecture**

### **How WebDriver Works**

Understanding the architecture helps debug issues and optimize tests:

```
┌─────────────────────────────────────────────────────────────────┐
│                    WEBDRIVER ARCHITECTURE                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│   TEST SCRIPT (Python/Java/etc)                                  │
│   ┌─────────────────────────────────────┐                       │
│   │ from selenium import webdriver      │                       │
│   │ driver = webdriver.Chrome()         │                       │
│   │ driver.get("https://example.com")   │                       │
│   └─────────────┬───────────────────────┘                       │
│                 │                                                │
│                 ▼ HTTP/JSON or W3C Protocol                      │
│   ┌─────────────────────────────────────┐                       │
│   │   BROWSER DRIVER (ChromeDriver/     │◄── Executable file    │
│   │    GeckoDriver/EdgeDriver/etc)      │    specific to OS     │
│   │                                     │                       │
│   │   • Starts browser process          │                       │
│   │   • Translates commands to browser  │                       │
│   │   • Returns results to script       │                       │
│   └─────────────┬───────────────────────┘                       │
│                 │                                                │
│                 ▼ Native Protocol (DevTools/MS COM/CDP)         │
│   ┌─────────────────────────────────────┐                       │
│   │       BROWSER (Chrome/Firefox/      │                       │
│   │        Edge/Safari)                 │                       │
│   │                                     │                       │
│   │   • Renders web page                │                       │
│   │   • Executes JavaScript             │                       │
│   │   • Returns DOM state               │                       │
│   └─────────────────────────────────────┘                       │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### **Communication Protocols**

**1. JSON Wire Protocol (Legacy)**
- Selenium 3.x and earlier
- RESTful API over HTTP
- Commands sent as JSON payloads

**2. W3C WebDriver Protocol (Current Standard)**
- Selenium 4.x default
- Standardized by W3C
- More efficient, better error handling
- Same endpoints but standardized format

```python
# Example of what happens when you call driver.find_element()
# Under the hood (simplified):

# 1. Your script sends HTTP POST to:
#    http://localhost:9515/session/{session_id}/element

# 2. JSON payload:
{
    "using": "css selector",
    "value": "#username"
}

# 3. ChromeDriver translates to Chrome DevTools Protocol:
#    DOM.querySelector

# 4. Chrome returns element ID

# 5. WebDriver returns WebElement object to your script
```

### **Selenium 4 BiDirectional (BiDi) Protocol**

**New in Selenium 4:** BiDirectional communication allows the browser to push events to the test script, not just respond to requests.

**Use Cases:**
- Network interception
- Console log capture
- Performance metrics
- JavaScript dialog handling

```python
# Selenium 4 BiDi example (Chrome DevTools Protocol)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

driver = webdriver.Chrome()

# Enable Network monitoring
driver.execute_cdp_cmd('Network.enable', {})

# Listen for network events
logs = driver.get_log('performance')  # Capture network activity

# Intercept requests (advanced)
driver.execute_cdp_cmd('Fetch.enable', {
    'patterns': [{'urlPattern': '*.js'}]
})
```

---

## **16.3 Installation and Setup**

### **Prerequisites**

1. **Programming Language** (Python recommended for beginners)
2. **Browser** (Chrome, Firefox, Edge, or Safari)
3. **Browser Driver** (ChromeDriver, GeckoDriver, etc.)
4. **Selenium Library** (language-specific binding)

### **Step-by-Step Setup**

#### **Python Setup**

```bash
# 1. Install Python (3.8+ recommended)
python --version  # Verify installation

# 2. Create virtual environment (recommended)
python -m venv selenium_env
source selenium_env/bin/activate  # Linux/Mac
# or
selenium_env\Scripts\activate  # Windows

# 3. Install Selenium
pip install selenium

# 4. Install WebDriver Manager (automatic driver management)
pip install webdriver-manager

# 5. Verify installation
python -c "import selenium; print(selenium.__version__)"
```

#### **Java Setup**

```xml
<!-- Maven dependency (pom.xml) -->
<dependency>
    <groupId>org.seleniumhq.selenium</groupId>
    <artifactId>selenium-java</artifactId>
    <version>4.15.0</version>
</dependency>

<!-- WebDriverManager for automatic driver management -->
<dependency>
    <groupId>io.github.bonigarcia</groupId>
    <artifactId>webdrivermanager</artifactId>
    <version>5.6.2</version>
</dependency>
```

### **Driver Management Strategies**

**Option 1: WebDriver Manager (Recommended)**
Automatic download and cache of correct driver version.

```python
# Python with WebDriver Manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager

# Chrome - Automatic
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Firefox - Automatic
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()))
```

**Option 2: Manual Driver Management**
Download drivers manually and place in PATH.

```python
# Manual path specification
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Explicit driver path
service = Service('/path/to/chromedriver')  # Linux/Mac
# service = Service('C:\\drivers\\chromedriver.exe')  # Windows

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
```

**Driver Download Locations:**
- ChromeDriver: https://chromedriver.chromium.org/downloads
- GeckoDriver (Firefox): https://github.com/mozilla/geckodriver/releases
- EdgeDriver: https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
- SafariDriver: Built into macOS (enable in Safari > Preferences > Advanced)

---

## **16.4 Locating Web Elements**

Finding elements is the foundation of Selenium automation. Choosing the right strategy determines test stability and maintainability.

### **Location Strategy Hierarchy (Best to Worst)**

```
1. ID (Fastest, most stable)
   └── <input id="username">
   
2. Name (Good for forms)
   └── <input name="email">
   
3. CSS Selector (Flexible, fast)
   └── #login-form .submit-btn
   
4. XPath (Powerful, slower)
   └── //input[@type='text']
   
5. Class Name (Risky, often not unique)
   └── btn-primary (may appear multiple times)
   
6. Tag Name (Too generic)
   └── div, input, button
   
7. Link Text (Only for links)
   └── Click Here
   
8. Partial Link Text (Fragile)
   └── Click
```

### **16.4.1 By ID**

The **ID** attribute should be unique within the page. Fastest and most reliable.

```python
from selenium.webdriver.common.by import By

# HTML: <input id="username" type="text">
element = driver.find_element(By.ID, "username")

# Multiple elements with same ID (invalid HTML but happens)
elements = driver.find_elements(By.ID, "username")  # Returns list
```

**Java Equivalent:**
```java
WebElement element = driver.findElement(By.id("username"));
```

### **16.4.2 By Name**

Common for form elements. Not required to be unique.

```python
# HTML: <input name="email" type="email">
email_field = driver.find_element(By.NAME, "email")

# HTML: <input name="newsletter" type="checkbox" value="yes">
checkbox = driver.find_element(By.NAME, "newsletter")
```

### **16.4.3 By Class Name**

**Warning:** Often not unique. Returns first match only.

```python
# HTML: <button class="btn btn-primary submit-btn">
button = driver.find_element(By.CLASS_NAME, "submit-btn")

# Finds element with class "submit-btn" (not full class string)
# Wrong: driver.find_element(By.CLASS_NAME, "btn btn-primary")  # Spaces not allowed
# Right: Use CSS selector for multiple classes
button = driver.find_element(By.CSS_SELECTOR, ".btn.btn-primary")
```

### **16.4.4 CSS Selectors**

**CSS Selectors** are patterns used to select elements based on their attributes, structure, and state.

**Basic CSS Selectors:**

| **Selector** | **Description** | **Example** |
|--------------|----------------|-------------|
| `#id` | ID selector | `#username` |
| `.class` | Class selector | `.btn-primary` |
| `tag` | Tag selector | `input`, `div` |
| `[attr=value]` | Attribute selector | `[type="text"]` |
| `A B` | Descendant (space) | `div input` |
| `A > B` | Direct child | `form > input` |
| `A + B` | Adjacent sibling | `h1 + p` |
| `A ~ B` | General sibling | `h1 ~ p` |

```python
# ID
driver.find_element(By.CSS_SELECTOR, "#username")

# Class
driver.find_element(By.CSS_SELECTOR, ".login-button")

# Tag + Class
driver.find_element(By.CSS_SELECTOR, "button.login-btn")

# Attribute
driver.find_element(By.CSS_SELECTOR, "[data-testid='submit-btn']")
driver.find_element(By.CSS_SELECTOR, "input[type='password']")

# Hierarchy
driver.find_element(By.CSS_SELECTOR, "#login-form input")  # Descendant
driver.find_element(By.CSS_SELECTOR, "nav > ul > li")      # Direct children

# Multiple classes
driver.find_element(By.CSS_SELECTOR, ".btn.btn-large.btn-primary")

# Pseudo-classes
driver.find_element(By.CSS_SELECTOR, "input:focus")
driver.find_element(By.CSS_SELECTOR, "button:disabled")
driver.find_element(By.CSS_SELECTOR, "a:hover")

# Complex combinations
driver.find_element(By.CSS_SELECTOR, "div.container > form#login input[type='text']")
```

### **16.4.5 XPath**

**XPath (XML Path Language)** is powerful but slower than CSS. Use when CSS can't locate element.

**XPath Types:**

**1. Absolute XPath (Fragile - Avoid)**
```python
# Starts from root (html)
# Breaks with any DOM change
driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/form/input[1]")
```

**2. Relative XPath (Preferred)**
```python
# Starts with // (anywhere in document)
driver.find_element(By.XPATH, "//input[@id='username']")
```

**XPath Axes and Functions:**

```python
# Basic syntax: //tag[@attribute='value']
driver.find_element(By.XPATH, "//input[@id='username']")
driver.find_element(By.XPATH, "//button[text()='Submit']")

# Contains (partial match)
driver.find_element(By.XPATH, "//input[contains(@id, 'name')]")
driver.find_element(By.XPATH, "//button[contains(text(), 'Save')]")

# Starts-with
driver.find_element(By.XPATH, "//input[starts-with(@id, 'user')]")

# Multiple attributes (and)
driver.find_element(By.XPATH, "//input[@type='text' and @class='form-control']")

# Or condition
driver.find_element(By.XPATH, "//input[@id='username' or @name='username']")

# Parent/Child relationships
driver.find_element(By.XPATH, "//div[@class='form-group']/input")  # Direct child
driver.find_element(By.XPATH, "//div[@class='form-group']//input")  # Any descendant

# Parent axis (go up)
driver.find_element(By.XPATH, "//input[@id='child']/parent::div")
driver.find_element(By.XPATH, "//input[@id='child']/..")  # Shorthand

# Following-sibling
driver.find_element(By.XPATH, "//label[@for='username']/following-sibling::input")

# Preceding-sibling
driver.find_element(By.XPATH, "//input[@id='submit']/preceding-sibling::input")

# Ancestor (any parent up to html)
driver.find_element(By.XPATH, "//input[@id='username']/ancestor::form")

# Position/index (1-based)
driver.find_element(By.XPATH, "(//input[@class='form-control'])[1]")  # First
driver.find_element(By.XPATH, "(//input[@class='form-control'])[last()]")  # Last

# XPath functions
driver.find_element(By.XPATH, "//input[string-length(@value) > 5]")
driver.find_element(By.XPATH, "//div[count(.//input) > 3]")  # Div with >3 inputs
```

### **16.4.6 Relative Locators (Selenium 4)**

**New in Selenium 4:** Locate elements based on their position relative to other elements.

```python
from selenium.webdriver.support.relative_locator import locate_with
from selenium.webdriver.common.by import By

# Find element above another element
password_field = driver.find_element(By.ID, "password")
username_field = driver.find_element(locate_with(By.TAG_NAME, "input").above(password_field))

# Find element below
submit_btn = driver.find_element(By.ID, "submit")
terms_checkbox = driver.find_element(locate_with(By.TAG_NAME, "input").below(submit_btn))

# Find left of / right of
cancel_btn = driver.find_element(By.ID, "cancel")
submit_btn = driver.find_element(locate_with(By.TAG_NAME, "button").to_the_right_of(cancel_btn))

# Near (within 50 pixels)
help_icon = driver.find_element(By.ID, "help")
tooltip = driver.find_element(locate_with(By.CLASS_NAME, "tooltip").near(help_icon))
```

---

## **16.5 WebDriver Commands and Methods**

### **Browser Navigation**

```python
from selenium import webdriver

driver = webdriver.Chrome()

# Open URL
driver.get("https://www.example.com")

# Get current URL
current_url = driver.current_url
assert "example.com" in current_url

# Get page title
title = driver.title
assert "Example Domain" in title

# Get page source (HTML)
page_source = driver.page_source

# Navigation commands
driver.back()      # Browser back button
driver.forward()   # Browser forward button
driver.refresh()   # Refresh page

# Window management
driver.maximize_window()
driver.minimize_window()
driver.fullscreen_window()

# Get window size/position
size = driver.get_window_size()  # {'width': 1920, 'height': 1080}
driver.set_window_size(1024, 768)

position = driver.get_window_position()
driver.set_window_position(0, 0)

# Selenium 4: New Window/Tab management
driver.switch_to.new_window('tab')  # Open new tab
driver.switch_to.new_window('window')  # Open new window

# Close current window/tab
driver.close()

# Quit browser (all windows)
driver.quit()
```

### **Element Interaction**

```python
from selenium.webdriver.common.by import By

# Finding elements
element = driver.find_element(By.ID, "username")  # Single element
elements = driver.find_elements(By.CLASS_NAME, "btn")  # List of elements

# Text input
element.clear()  # Clear existing text
element.send_keys("testuser")  # Type text
element.send_keys(" appended")  # Append to existing

# Special keys
from selenium.webdriver.common.keys import Keys
element.send_keys(Keys.RETURN)  # Press Enter
element.send_keys(Keys.TAB)     # Press Tab
element.send_keys(Keys.CONTROL, 'a')  # Ctrl+A (Select all)
element.send_keys(Keys.CONTROL, 'c')  # Ctrl+C (Copy)

# Clicking
button = driver.find_element(By.ID, "submit")
button.click()

# Selenium 4: Click with options (coordinates, button type)
from selenium.webdriver.common.action_chains import ActionChains
ActionChains(driver).click(button).perform()

# Getting element properties
text = element.text  # Visible text content
tag_name = element.tag_name  # "input", "div", etc.

# Attributes
attribute_value = element.get_attribute("value")  # Input value
attribute_value = element.get_attribute("href")   # Link URL
attribute_value = element.get_attribute("class")  # Class names
attribute_value = element.get_attribute("innerHTML")  # HTML content

# CSS properties
color = element.value_of_css_property("color")
font_size = element.value_of_css_property("font-size")

# Element state
is_displayed = element.is_displayed()  # Visible on page
is_enabled = element.is_enabled()      # Not disabled
is_selected = element.is_selected()    # Checkbox/radio selected

# Location and size
location = element.location  # {'x': 100, 'y': 200}
size = element.size          # {'width': 150, 'height': 40}
```

---

## **16.6 Handling Different Web Elements**

### **Text Fields and Text Areas**

```python
# Standard text input
username = driver.find_element(By.ID, "username")
username.clear()
username.send_keys("testuser")

# Text area
description = driver.find_element(By.ID, "description")
description.clear()
description.send_keys("This is a long description...\nWith multiple lines.")

# Password field (masked input)
password = driver.find_element(By.ID, "password")
password.send_keys("secret123")

# Email field (with validation)
email = driver.find_element(By.ID, "email")
email.send_keys("test@example.com")

# Number field
age = driver.find_element(By.ID, "age")
age.clear()
age.send_keys("25")

# Date picker (HTML5)
date_field = driver.find_element(By.ID, "dob")
# Method 1: Send date string in expected format
date_field.send_keys("12/25/1990")
# Method 2: Use JavaScript to set value
driver.execute_script("arguments[0].value = '1990-12-25';", date_field)
```

### **Buttons**

```python
# Standard button
submit_btn = driver.find_element(By.ID, "submit")
submit_btn.click()

# Button inside form (type="submit")
driver.find_element(By.XPATH, "//button[@type='submit']").click()

# Disabled button check
btn = driver.find_element(By.ID, "save")
if btn.is_enabled():
    btn.click()
else:
    print("Button is disabled")

# JavaScript click (when regular click doesn't work)
driver.execute_script("arguments[0].click();", btn)

# Double click
from selenium.webdriver.common.action_chains import ActionChains
ActionChains(driver).double_click(btn).perform()

# Right click (context click)
ActionChains(driver).context_click(btn).perform()
```

### **Dropdowns (Select Class)**

```python
from selenium.webdriver.support.ui import Select

# HTML: <select id="country">
#   <option value="us">United States</option>
#   <option value="uk">United Kingdom</option>
# </select>

dropdown = Select(driver.find_element(By.ID, "country"))

# Select by visible text
dropdown.select_by_visible_text("United States")

# Select by value attribute
dropdown.select_by_value("us")

# Select by index (0-based)
dropdown.select_by_index(0)

# Get selected option
selected = dropdown.first_selected_option
print(selected.text)  # "United States"

# Multi-select dropdown
multi = Select(driver.find_element(By.ID, "languages"))
multi.select_by_visible_text("English")
multi.select_by_visible_text("Spanish")

# Deselect (for multi-select only)
multi.deselect_all()
multi.deselect_by_visible_text("Spanish")

# Check if multi-select
is_multi = multi.is_multiple  # True/False

# Get all options
all_options = multi.options  # List of WebElements
for option in all_options:
    print(option.text)
```

### **Checkboxes**

```python
# Single checkbox
checkbox = driver.find_element(By.ID, "newsletter")

# Check (if not already checked)
if not checkbox.is_selected():
    checkbox.click()

# Uncheck (if checked)
if checkbox.is_selected():
    checkbox.click()

# Toggle using JavaScript (more reliable)
driver.execute_script("arguments[0].checked = true;", checkbox)

# Multiple checkboxes with same name
checkboxes = driver.find_elements(By.NAME, "interests")
for checkbox in checkboxes:
    if checkbox.get_attribute("value") == "sports":
        checkbox.click()
        break
```

### **Radio Buttons**

```python
# Radio button group (same name, different values)
# HTML: <input type="radio" name="gender" value="male">
#       <input type="radio" name="gender" value="female">

# Select by value
male_radio = driver.find_element(By.CSS_SELECTOR, "input[name='gender'][value='male']")
male_radio.click()

# Or find all and select by index
genders = driver.find_elements(By.NAME, "gender")
genders[0].click()  # Select first (male)

# Verify selection
assert male_radio.is_selected()
```

### **Links**

```python
# Find by link text (exact match)
link = driver.find_element(By.LINK_TEXT, "Click here to login")

# Find by partial link text
link = driver.find_element(By.PARTIAL_LINK_TEXT, "login")

# Get href attribute
url = link.get_attribute("href")
print(url)  # https://example.com/login

# Click
link.click()

# Verify link opens in new tab (target="_blank")
assert link.get_attribute("target") == "_blank"
```

### **File Upload**

```python
# Standard file input (type="file")
# Note: Cannot use Selenium to interact with OS file dialog
# Must send file path directly to input element

file_input = driver.find_element(By.ID, "file-upload")
file_input.send_keys("/path/to/file.pdf")  # Absolute path

# For Windows (escape backslashes or use raw string)
file_input.send_keys(r"C:\Users\test\file.pdf")
# or
file_input.send_keys("C:\\Users\\test\\file.pdf")

# Then submit form
driver.find_element(By.ID, "upload-btn").click()
```

### **JavaScript Alerts, Confirmations, Prompts**

```python
from selenium.webdriver.common.alert import Alert

# Wait for alert to appear
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Simple Alert (OK only)
WebDriverWait(driver, 10).until(EC.alert_is_present())
alert = driver.switch_to.alert
print(alert.text)  # Get alert text
alert.accept()     # Click OK

# Confirmation (OK/Cancel)
confirm = driver.switch_to.alert
print(confirm.text)
confirm.accept()   # Click OK
# confirm.dismiss()  # Click Cancel

# Prompt (Input text)
prompt = driver.switch_to.alert
prompt.send_keys("Input text")  # Type in prompt
prompt.accept()

# Switch back to main content
driver.switch_to.default_content()
```

### **Frames and iFrames**

```python
# Switch to frame by index (0-based)
driver.switch_to.frame(0)

# Switch to frame by name or id
driver.switch_to.frame("frame-name")
driver.switch_to.frame("frame-id")

# Switch to frame by WebElement
frame_element = driver.find_element(By.ID, "frame-id")
driver.switch_to.frame(frame_element)

# Work with elements inside frame
driver.find_element(By.ID, "inside-frame").click()

# Switch back to parent frame
driver.switch_to.parent_frame()

# Switch to main document (out of all frames)
driver.switch_to.default_content()

# Nested frames (switch one by one)
driver.switch_to.frame("outer-frame")
driver.switch_to.frame("inner-frame")
# ... do work ...
driver.switch_to.default_content()  # Goes directly to main
```

---

## **16.7 Waits and Synchronization**

**The #1 cause of flaky tests:** Timing issues. Selenium executes faster than page loads.

### **Types of Waits**

```
Wait Strategy Decision Tree:

Is element immediately available?
├── Yes → Use Implicit Wait (global setting)
│
└── No → Is condition predictable?
    ├── Yes → Use Explicit Wait (WebDriverWait)
    │           └── Wait for specific condition
    │
    └── No → Use Fluent Wait (polling + exceptions)
                └── Custom polling frequency, ignore exceptions
```

### **16.7.1 Implicit Wait**

Sets a global timeout for all `find_element` calls. If element not found immediately, Selenium polls the DOM until timeout.

```python
# Set once per session (applies to all subsequent find_element calls)
driver.implicitly_wait(10)  # 10 seconds

# Now all find_element calls wait up to 10 seconds
element = driver.find_element(By.ID, "dynamic-element")  # Waits if not present

# Disadvantages:
# - Applies to ALL find operations (even if element clearly doesn't exist)
# - Can slow down tests when elements genuinely missing
# - No polling control
```

### **16.7.2 Explicit Wait (Recommended)**

Wait for specific condition using `WebDriverWait`.

```python
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Basic explicit wait
wait = WebDriverWait(driver, 10)  # 10 second timeout
element = wait.until(EC.presence_of_element_located((By.ID, "dynamic-element")))

# Common Expected Conditions
wait = WebDriverWait(driver, 10)

# Presence (in DOM, not necessarily visible)
element = wait.until(EC.presence_of_element_located((By.ID, "id")))

# Visibility (in DOM AND visible - not just display:none, but size > 0)
element = wait.until(EC.visibility_of_element_located((By.ID, "id")))

# Clickable (visible AND enabled)
element = wait.until(EC.element_to_be_clickable((By.ID, "id")))

# Invisibility (wait for loading spinner to disappear)
wait.until(EC.invisibility_of_element_located((By.ID, "loading")))

# Text present in element
wait.until(EC.text_to_be_present_in_element((By.ID, "status"), "Complete"))

# Element selection state
wait.until(EC.element_to_be_selected(driver.find_element(By.ID, "checkbox")))

# Alert present
wait.until(EC.alert_is_present())

# Title contains
wait.until(EC.title_contains("Success"))

# URL contains
wait.until(EC.url_contains("/dashboard"))

# Custom condition (lambda)
wait.until(lambda driver: driver.find_element(By.ID, "counter").text == "100")
```

**Complete List of Expected Conditions:**

```python
# Element state
presence_of_element_located(locator)
visibility_of(element)
visibility_of_element_located(locator)
invisibility_of_element_located(locator)
element_to_be_clickable(locator)
element_to_be_selected(element)
element_located_to_be_selected(locator)
staleness_of(element)  # Element removed from DOM

# Text
text_to_be_present_in_element(locator, text)
text_to_be_present_in_element_value(locator, text)

# Window/Frame
frame_to_be_available_and_switch_to_it(locator)
new_window_is_opened(current_handles)
number_of_windows_to_be(num)

# Alert
alert_is_present()

# URL/Title
title_is(title)
title_contains(title)
url_to_be(url)
url_contains(url)
url_matches(pattern)

# Logic
presence_of_all_elements_located(locator)
visibility_of_any_elements_located(locator)
visibility_of_all_elements_located(locator)
```

### **16.7.3 Fluent Wait**

Advanced wait with polling frequency and exception ignoring.

```python
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

# Fluent wait configuration
wait = WebDriverWait(
    driver,
    timeout=30,           # Max wait time
    poll_frequency=2,     # Check every 2 seconds (default is 0.5)
    ignored_exceptions=[NoSuchElementException, ElementNotInteractableException]
)

# Use like explicit wait
element = wait.until(EC.element_to_be_clickable((By.ID, "slow-button")))
```

### **Wait Best Practices**

```python
# BAD: Hardcoded sleep (always waits full time)
import time
time.sleep(5)  # Never do this!

# GOOD: Explicit wait for specific condition
wait = WebDriverWait(driver, 10)
element = wait.until(EC.element_to_be_clickable((By.ID, "submit")))

# BETTER: Page Object with built-in waits
class LoginPage:
    def __init__(self, driver):
        self.driver = driver
        self.wait = WebDriverWait(driver, 10)
    
    def enter_username(self, username):
        # Wait for element ready before interaction
        field = self.wait.until(EC.visibility_of_element_located((By.ID, "username")))
        field.clear()
        field.send_keys(username)
        return self
    
    def click_submit(self):
        # Wait for clickable ensures not disabled/loading
        btn = self.wait.until(EC.element_to_be_clickable((By.ID, "submit")))
        btn.click()
```

---

## **16.8 Advanced Interactions**

### **ActionChains**

Complex user interactions (mouse movements, drag-drop, keyboard shortcuts).

```python
from selenium.webdriver.common.action_chains import ActionChains

actions = ActionChains(driver)

# Hover (mouse over)
menu = driver.find_element(By.ID, "main-menu")
actions.move_to_element(menu).perform()

# Hover then click submenu
submenu = driver.find_element(By.ID, "submenu-item")
actions.move_to_element(menu).pause(1).click(submenu).perform()

# Click and hold, move, release (drag and drop)
source = driver.find_element(By.ID, "draggable")
target = driver.find_element(By.ID, "droppable")
actions.drag_and_drop(source, target).perform()

# Or manual drag-drop
actions.click_and_hold(source).move_to_element(target).release().perform()

# Right-click
element = driver.find_element(By.ID, "context-menu-area")
actions.context_click(element).perform()

# Double-click
actions.double_click(element).perform()

# Click with offset (specific coordinates within element)
actions.move_to_element_with_offset(element, 10, 10).click().perform()

# Key combinations
search_box = driver.find_element(By.ID, "search")
actions.click(search_box).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).send_keys(Keys.DELETE).perform()

# Send keys to active element
actions.send_keys(Keys.TAB).pause(1).send_keys("text").perform()

# Scroll to element
actions.move_to_element(element).perform()  # Selenium 4 scrolls automatically

# Scroll by offset
actions.scroll_by_amount(0, 500).perform()  # Scroll down 500px
actions.scroll_from_origin(element, 0, 100).perform()  # Scroll within element
```

### **JavaScript Execution**

When Selenium methods fail, execute JavaScript directly.

```python
# Execute JavaScript
driver.execute_script("console.log('Hello from Selenium')")

# Execute with arguments
element = driver.find_element(By.ID, "username")
driver.execute_script("arguments[0].value = 'set via JS';", element)

# Get return value
title = driver.execute_script("return document.title;")
element_text = driver.execute_script("return arguments[0].textContent;", element)

# Scroll to element
driver.execute_script("arguments[0].scrollIntoView(true);", element)

# Scroll to bottom of page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Click via JS (bypasses visibility checks - use carefully)
driver.execute_script("arguments[0].click();", element)

# Remove attribute (e.g., make hidden input visible for testing)
driver.execute_script("arguments[0].removeAttribute('hidden');", element)

# Set inner HTML
driver.execute_script("arguments[0].innerHTML = '<b>Bold Text</b>';", element)

# Handle Shadow DOM (Selenium 4 has built-in support, but JS works too)
shadow_host = driver.find_element(By.ID, "shadow-host")
shadow_root = driver.execute_script("return arguments[0].shadowRoot", shadow_host)
element_in_shadow = shadow_root.find_element(By.CSS_SELECTOR, "#inside-shadow")
```

### **Handling Shadow DOM**

```python
# Selenium 4 Shadow DOM support
shadow_host = driver.find_element(By.CSS_SELECTOR, "#shadow-host")
shadow_root = shadow_host.shadow_root  # Access shadow root
element = shadow_root.find_element(By.CSS_SELECTOR, "#inside-shadow")

# Or using JavaScript (works in all versions)
element = driver.execute_script("""
    return document.querySelector('#shadow-host')
           .shadowRoot.querySelector('#inside-shadow');
""")
```

### **Taking Screenshots**

```python
# Full page screenshot
driver.save_screenshot("full_page.png")

# Specific element screenshot (Selenium 4)
element = driver.find_element(By.ID, "login-form")
element.screenshot("element.png")

# Screenshot as base64 (for embedding in reports)
screenshot_base64 = driver.get_screenshot_as_base64()

# Screenshot as bytes (for PIL processing)
screenshot_bytes = driver.get_screenshot_as_png()

# Full page screenshot (Chrome DevTools protocol - Selenium 4 CDP)
driver.execute_cdp_cmd('Page.captureScreenshot', {'format': 'png', 'fullPage': True})
```

### **Handling Multiple Windows/Tabs**

```python
# Get current window handle
main_window = driver.current_window_handle

# Get all window handles
all_windows = driver.window_handles  # List of handles

# Open new tab (Selenium 4)
driver.switch_to.new_window('tab')

# Open new window
driver.switch_to.new_window('window')

# Switch to new window (after click opens new window)
for handle in driver.window_handles:
    if handle != main_window:
        driver.switch_to.window(handle)
        break

# Work in new window
assert "New Page" in driver.title

# Close new window and switch back
driver.close()
driver.switch_to.window(main_window)

# Or close all other windows
for handle in driver.window_handles:
    if handle != main_window:
        driver.switch_to.window(handle)
        driver.close()
driver.switch_to.window(main_window)
```

---

## **16.9 Cross-Browser Testing**

### **Browser-Specific Setup**

```python
# Chrome Options
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service

chrome_options = ChromeOptions()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--headless")  # Run without UI (CI/CD)
chrome_options.add_argument("--disable-gpu")  # For headless
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--no-sandbox")  # Required for Docker/Linux

# Preferences
prefs = {
    "download.default_directory": "/path/to/downloads",
    "download.prompt_for_download": False,
    "profile.default_content_setting_values.notifications": 2  # Block notifications
}
chrome_options.add_experimental_option("prefs", prefs)

# Exclude automation switches (avoid detection)
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
chrome_options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(options=chrome_options)

# Firefox Options
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService

firefox_options = FirefoxOptions()
firefox_options.add_argument("--width=1920")
firefox_options.add_argument("--height=1080")
firefox_options.headless = True  # Headless mode

# Firefox Profile
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
profile = FirefoxProfile()
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.manager.showWhenStarting", False)
firefox_options.profile = profile

driver = webdriver.Firefox(options=firefox_options)

# Edge Options (similar to Chrome)
from selenium.webdriver.edge.options import Options as EdgeOptions
edge_options = EdgeOptions()
edge_options.add_argument("--start-maximized")
driver = webdriver.Edge(options=edge_options)

# Safari (macOS only)
# Enable Develop menu in Safari Preferences first
driver = webdriver.Safari()
```

### **Cross-Browser Test Execution**

```python
# Parameterized cross-browser testing (pytest)
import pytest
from selenium import webdriver

@pytest.fixture(params=["chrome", "firefox", "edge"])
def driver(request):
    browser = request.param
    
    if browser == "chrome":
        driver = webdriver.Chrome()
    elif browser == "firefox":
        driver = webdriver.Firefox()
    elif browser == "edge":
        driver = webdriver.Edge()
    
    driver.maximize_window()
    yield driver
    driver.quit()

def test_login_cross_browser(driver):
    """Runs on Chrome, Firefox, and Edge"""
    driver.get("https://example.com/login")
    # ... test steps
```

---

## **16.10 Best Practices and Tips**

### **1. Use Page Object Model (POM)**

Never put locators directly in test methods. See Chapter 12 for detailed POM implementation.

### **2. Prefer Explicit Waits**

```python
# Good
wait = WebDriverWait(driver, 10)
element = wait.until(EC.element_to_be_clickable((By.ID, "submit")))

# Bad
time.sleep(5)
element = driver.find_element(By.ID, "submit")
```

### **3. Stable Locators**

Use data attributes for testing:

```html
<!-- Good for automation -->
<button data-testid="submit-button">Submit</button>

<!-- Bad - changes with styling -->
<button class="btn btn-primary btn-lg">Submit</button>
```

```python
driver.find_element(By.CSS_SELECTOR, "[data-testid='submit-button']")
```

### **4. Handle Exceptions Gracefully**

```python
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    ElementNotInteractableException
)

try:
    element = driver.find_element(By.ID, "optional-element")
except NoSuchElementException:
    # Element doesn't exist - might be OK
    pass

try:
    wait.until(EC.element_to_be_clickable((By.ID, "slow-button"))).click()
except TimeoutException:
    # Take screenshot for debugging
    driver.save_screenshot("timeout_error.png")
    raise
```

### **5. Clean Up Resources**

```python
try:
    driver = webdriver.Chrome()
    # ... tests ...
finally:
    driver.quit()  # Always close browser
```

### **6. Use Relative URLs**

```python
# Configurable base URL
BASE_URL = "https://staging.example.com"  # Change for different environments

driver.get(f"{BASE_URL}/login")
# Instead of hardcoded:
# driver.get("https://staging.example.com/login")
```

### **7. Avoid Thread.sleep()**

Always use Selenium waits instead of `time.sleep()`.

### **8. Test Data Management**

Don't hardcode test data in tests. Use fixtures, external files, or factories.

### **9. Headless for CI**

Run headless in CI/CD for speed and stability:

```python
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
```

### **10. Logging**

Enable Selenium logging for debugging:

```python
import logging
logging.basicConfig(level=logging.INFO)

# Or specific WebDriver logging
service = Service(log_path="selenium.log")
driver = webdriver.Chrome(service=service)
```

---

## **Chapter Summary**

### **Key Takeaways:**

1. **Architecture:** Selenium WebDriver communicates via HTTP/JSON (W3C protocol) to browser-specific drivers (ChromeDriver, GeckoDriver), which control the browser using native protocols.

2. **Setup:** Use WebDriverManager for automatic driver management. Set up virtual environments and maintain driver-browser version compatibility.

3. **Locators:** Prefer ID > Name > CSS Selector > XPath. CSS is faster than XPath. Use attributes like `data-testid` for stable automation.

4. **Element Interaction:** Use `send_keys()` for input, `click()` for interaction. Use Select class for dropdowns. Handle alerts with `switch_to.alert`.

5. **Synchronization:** Never use `time.sleep()`. Use Explicit Waits (`WebDriverWait` + `ExpectedConditions`) for reliability. Implicit waits are global but less flexible.

6. **Advanced Actions:** Use ActionChains for hover, drag-drop, complex clicks. Execute JavaScript when standard methods fail (Shadow DOM, hidden elements).

7. **Cross-Browser:** Use Options classes to configure browser-specific settings. Parameterize tests to run on multiple browsers.

8. **Best Practices:** Implement Page Object Model, use stable locators, handle exceptions, clean up resources with `quit()`, and run headless in CI.

### **Common Exceptions and Solutions:**

| **Exception** | **Cause** | **Solution** |
|---------------|-----------|--------------|
| `NoSuchElementException` | Element not in DOM | Check locator, add wait |
| `TimeoutException` | Wait condition not met | Increase timeout, check condition |
| `ElementNotInteractableException` | Element present but not clickable | Wait for visibility/clickability, scroll into view |
| `StaleElementReferenceException` | DOM changed after finding element | Re-find element before interaction |
| `NoAlertPresentException` | Alert not present when expected | Add wait for alert |
| `WebDriverException` | Driver crashed/browser closed | Restart driver, check resources |

---

## **📖 Next Chapter: Chapter 17 - Modern Browser Automation Tools**

Now that you've mastered Selenium WebDriver—the foundation of web automation—**Chapter 17** introduces the next generation of browser automation tools that are gaining rapid industry adoption.

In **Chapter 17**, you'll explore:

- **Cypress:** The developer-friendly testing framework with time-travel debugging, automatic waiting, and real-time reloads
- **Playwright:** Microsoft's modern automation library with auto-waiting, parallel execution, and multi-browser support (Chromium, Firefox, WebKit) from a single API
- **Architecture Comparisons:** Selenium vs. Cypress vs. Playwright—when to use which
- **Cypress Deep Dive:** Commands, assertions, fixtures, custom commands, and CI/CD integration
- **Playwright Deep Dive:** Codegen (record/playback), trace viewer, parallel execution, and visual testing
- **Tool Selection Guide:** Decision matrix based on team skills, application type, and testing requirements

**Why Chapter 17 is Critical:** While Selenium remains the industry standard, Cypress and Playwright are disrupting the market with modern architectures that solve Selenium's pain points (flakiness, waits, debugging). Understanding all three tools makes you a versatile automation engineer capable of choosing the right tool for the job.

**Continue to Chapter 17 to learn the modern alternatives to Selenium and expand your automation toolkit!**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='15. web_application_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='17. modern_browser_automation_tools.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
